# Lab 8 : Sarcasm Detection

@copyright: 
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.

---


## 목차

1. Sarcasm Detection
2. Data Preprocessing
3. Modeling
4. Training / Evaluation

## 1. Sarcasm Detection

`Sarcasm Detection` task는 target 문장이 sarcastic한지 아닌지를 판별하는 task이다.

[ACL2020 dataset](https://github.com/EducationalTestingService/sarcasm)을 사용한다. 해당 dataset은 3개의 column으로 구성되어 있다.

- `label`: sarcastic 여부를 나타내는 binary value (NOT-SARCASM: non-sarcastic, SARCASM: sarcastic)
- `response`: target 문장
- `context`: target 문장 앞 문장들 (최대 20)

### 본 실습에서는, 지금까지 배운 내용을 활용하여 Dataset의 이해 및 전처리, 배운 Model의 Implement 및 비교, 그리고 학습 및 평가를 진행한다. 

- 원하는 전처리, 모델, 학습 방법을 적절히 선택하여 진행하고, 여러 실험을 통해 최적의 결과를 얻도록 한다.
- 그 과정에서 변경하는 모델의 구조 및 전처리 방법, 학습 전략, 하이퍼파라미터 등을 기록하고, 그에 따른 결과 및 해석을 함께 기록한다. (해당 기록을 평가) 
- 최종적으로, 75% 이상의 accuracy를 달성하도록 한다.
- 본 실습에서는 가장 간단한 BertForSequenceClassification 모델을 사용한다. 해당 코드를 참고하여 진행하면 된다.

In [ ]:
!git clone https://github.com/EducationalTestingService/sarcasm.git

## 2. Data Preprocessing

In [ ]:
import json
import pprint

with open("./sarcasm/twitter/sarcasm_detection_shared_task_twitter_training.jsonl", 'r', encoding="utf-8") as file:
    datalines = file.readlines()

data = []
for line in datalines:
    data.append(json.loads(line))

print("Data 개수: ", len(data))
print("Data 예시: ")
pprint.pprint(data[0])

In [ ]:
# Data 전처리. Data를 직접 살펴보며 Emoji, URL, Special Character 등 적절한 처리를 진행하도록 한다.
import json
import re
import emoji

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'@user', '', sentence)

    return sentence.strip()

# Example
print("Before: ", data[0]['response'])
print("After: ", preprocess_sentence(data[0]['response']))


In [ ]:
# data loader
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

class SarcasmDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        #원하는 Hyperparameter를 추가하여도 된다.
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        context = preprocess_sentence(
        response = preprocess_sentence(
        label = 

        #적절한 Context와 Response를 결합하여 Input으로 사용하도록 한다. Context 길이 또한 고려하여야하는 Hyperparameter이다.
        input = 

        # Tokenizing
        input_ids = self.tokenizer.encode(input)
        token_type_ids = 
        attention_mask = 

        return input_ids, token_type_ids, attention_mask, label
    
# Example
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
dataset = SarcasmDataset(data, tokenizer, 256)

print("original_sentence", data[0]['context'][-1], data[0]['response'])
print("Input_ids: ", dataset[0][0])
print("Token_type_ids: ", dataset[0][1])
print("Attention_mask: ", dataset[0][2])
print("Label: ", dataset[0][3])

In [ ]:
# collate function considering padding
def collate_fn(batch):
    max_len = max(max(len(item[0]), len(item[1]), len(item[2])) for item in batch)  # Find max length in batch

    input_ids, token_type_ids, attention_mask, labels = [], [], [], []
    for item in batch:
        # Padding each sequence to the max length
        input_ids.append(item[0] + [0] * (max_len - len(item[0])))
        token_type_ids.append(item[1] + [0] * (max_len - len(item[1])))
        attention_mask.append(item[2] + [0] * (max_len - len(item[2])))
        labels.append(item[3])

    # Convert lists to tensors
    input_ids = torch.LongTensor(input_ids)
    token_type_ids = torch.LongTensor(token_type_ids)
    attention_mask = torch.LongTensor(attention_mask)
    labels = torch.LongTensor(labels)

    return input_ids, token_type_ids, attention_mask, labels

# Example
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
for x in dataloader:
    print("Input_ids: ", x[0])
    print("Token_type_ids: ", x[1])
    print("Attention_mask: ", x[2])
    print("Label: ", x[3])
    break

In [ ]:
# Train data
train_data = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Test data
with open("./sarcasm/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl", 'r', encoding="utf-8") as file:
    datalines = file.readlines()

test_data = []
for line in datalines:
    test_data.append(json.loads(line))

test_dataset = SarcasmDataset(test_data, tokenizer, 256)
test_data = DataLoader(test_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

## 3. Modeling

In [ ]:
from transformers import AutoModel, AutoConfig
import torch.nn as nn

class SarcasmDetectionModel(nn.Module):
    def __init__(self, model_name, num_labels):
        super(SarcasmDetectionModel, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
        self.encoder = AutoModel.from_pretrained(model_name, config=self.config)

        self.classifier = nn.Sequential(

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        outputs = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        pooled_output = #WHAT IS THIS?
        
        logits = self.classifier(pooled_output)

        return logits

# Example
model = SarcasmDetectionModel("bert-base-uncased", num_labels=2).cuda()

for x in train_data:
    input_ids, token_type_ids, attention_mask, labels = x
    logits = model(input_ids.cuda(), token_type_ids=token_type_ids.cuda(), attention_mask=attention_mask.cuda())
    print("Input_ids: ", input_ids)
    print("Token_type_ids: ", token_type_ids)
    print("Attention_mask: ", attention_mask)
    print("Logits: ", logits)
    print("Result: ", torch.argmax(logits, dim=-1))
    print("Label: ", labels)
    break

In [ ]:
# Optimizer, In here you can use any optimizer you want.
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-6)

# Loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Example
input_ids, token_type_ids, attention_mask, labels = next(iter(train_data))
output = model(input_ids.cuda(), token_type_ids.cuda(), attention_mask.cuda())
print("Output: ", output)
print("Gold: ", labels)

## 4. Training

In [ ]:
# Training
from tqdm import tqdm

epochs = 20
for epoch in range(epochs):
    model.train()
    for input_ids, token_type_ids, attention_mask, labels in tqdm(train_data):
        optimizer.zero_grad()

        output = model(input_ids.cuda(), token_type_ids.cuda(), attention_mask.cuda())
        loss = loss_fn(output, labels.cuda())
        loss.backward()
        optimizer.step()

    model.eval()
    total = 0
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for input_ids, token_type_ids, attention_mask, labels in tqdm(test_data):
        output = model(input_ids.cuda(), token_type_ids.cuda(), attention_mask.cuda())
        prediction = torch.argmax(output, dim=-1)
        tp += 
        fp += 
        fn += 
        tn += 
        total += len(labels)

    print("Accuracy: ", (tp + tn) / total)
    print("Precision: ", tp / (tp + fp))
    print("Recall: ", tp / (tp + fn))
    print("F1: ", 2 * tp / (2 * tp + fp + fn))